In [258]:
conda install geopandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.8.5

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /Users/user/anaconda3/envs/gee

  added / updated specs:
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boost-cpp-1.74.0           |       he5d75e3_0        15.6 MB  conda-forge
    cairo-1.16.0               |    ha8983da_1005         1.3 MB  conda-forge
    cfitsio-3.470              |       hdf94aef_6         1.3 MB  conda-forge
    curl-7.71.1                |       hcb81553_5         131 KB  conda-forge
    expat-2.2.9                |       hb1e8313_2         127 KB  conda-forge
    fiona-1.8.17               |   py38hcfd527e_0         938 KB  conda-forge
    fontconfig-2.13.1          |    h79c0d67_1002         273 KB  conda-f

In [207]:
import geemap
import ee
import os
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/4AEOv5KXFS-_bLaGu0X9whJUtA-4iyClo8QuHoMKhEmA6CeB-XbQwb8

Successfully saved authorization token.


In [187]:
#Display simple map
Map = geemap.Map () 
Map


Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [188]:
#choose which basemap to use... default is google basemap
Map.add_basemap("Google Maps")

In [249]:
Map.centerObject(ROI, 6)

In [190]:
# defining ROI
xmin,ymin,xmax,ymax = [122.7328125,72.187559318, 129.47779664, 73.97591048]

ROI = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

In [191]:
#import Landsat 8 images between june and August 2018 falling within ROI
images2018 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
         .filterBounds(ROI).filterDate('2018-06-01', '2018-08-31')

In [192]:
gsw = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').clip(ROI);
occurrence = gsw.select('occurrence')
#Map.addLayer(occurrence,'Water Occurrence (1984-2015)', True, 0.5);

In [193]:
#Visualization params
VIS_OCCURRENCE = {
        'min': 0,
        'max': 100,
        'palette': ['red', 'blue']}


In [194]:
occ = occurrence.updateMask(occurrence.divide(100))

In [195]:
Map.addLayer(occ,VIS_OCCURRENCE , 'occurrence', True, 0.5)

In [196]:
# --------Mask pixels with clouds and cloud shadows ---------

# Surface reflectance products come with a 'pixel_qa' band

# create function to mask clouds, cloud shadows, snow

def maskClouds(image):
  pixel_qa = image.select('pixel_qa')
  return image.updateMask(pixel_qa.eq(322)) # keep clear (0) and water (1) pixels  

# apply the masking function on the image collection
images2018_masked = images2018.map(maskClouds)

# visualize the first image in the collection, pre- and post- mask
visParams = {
    'bands': ['B4','B3','B2'], 
    'min': 0, 
    'max': 3000}

Map.addLayer(ee.Image(images2018_masked.first()), visParams, 'cloudsmasked_2018', False)
Map.addLayer(ee.Image(images2018.first()), visParams, 'original_2018', False)

In [197]:
#Calculate ndvi

def getNDVI(img):
  return img.addBands(img.normalizedDifference(ee.List(['B5', 'B4'])).rename('NDVI'))

In [198]:
#Visualization params for NDVI
ndviPalette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301'];
               

In [199]:
# "map" the NDVI band over image collection
ndvi2018 = images2018_masked.map(getNDVI);

In [200]:
# print one image to see the band is now there
#print(ee.Image(ndvi2018.first()));

In [201]:
#for each pixel, select the "best" set of bands from available images
#based on the maximum NDVI/greenness

#composite the image based on max NDVI
#this code creats a single image composit based on max NDVI value
composite = ndvi2018.qualityMosaic('NDVI').clip(ROI);
#print(composite);

In [202]:
#Visualize true color composite and compare with ndvi
Map.addLayer(composite.select('NDVI'),
            {'min':0, 'max': 1, 'palette': ndviPalette}, 'ndvi_2018');
Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'TC_composite_2018', False);



In [217]:
#/* color: #206f1b */
MESH =ee.FeatureCollection(
     [ee.Feature(

In [239]:
# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte()

# Paint all the polygon edges with the same number and 'width', display.
outline = empty.paint(**{
  'featureCollection': MESH,
  'color': 1,
  'width': 3
})



In [218]:
#merge classes
#classNames = geemMapee.FeatureCollection(MESH.merge(DSH).merge(GSSH).merge(WSMT).merge(DT).merger(PC).merge(SAND).merge(WATER))
#print (classNames)

In [219]:
#Map.addLayer(MESH, {'color': '#206f1b'}, 'MESH')
#Map.addLayer(DSH, {'color': '0000FF'}, '#777957')


In [221]:
Map.addLayer(image3, {
    'palette': ['000000', 'FF0000', '00FF00', '0000FF'],
    'max': 3,
    'opacity': 0.5
}, "MESH")

In [238]:
Map.centerObject(MESH1, 5)

In [237]:
MESH1 = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[126.35823708277854, 72.34141113708156],
                  [126.36141281825218, 72.3417235695464],
                  [126.36029701930198, 72.34232238347865],
                  [126.36055451136741, 72.34349391909498],
                  [126.36192780238304, 72.34474347414152],
                  [126.36544686061058, 72.34758068744891],
                  [126.3627861092678, 72.34791904122308],
                  [126.3613269875637, 72.34692998946569],
                  [126.35815125209007, 72.34487363120185],
                  [126.35737877589378, 72.34318151695544],
                  [126.35797959071311, 72.34164546193192]]]),
            {
              "landcover": 0,
              "system:index": "0"
            })]),

In [243]:
MT = {
  'reducer': ee.Reducer.mean(),
  'geometry': ee.Geometry.Polygon([
                  [126.35823708277854, 72.34141113708156],
                  [126.36141281825218, 72.3417235695464],
                  [126.36029701930198, 72.34232238347865],
                  [126.36055451136741, 72.34349391909498],
                  [126.36192780238304, 72.34474347414152],
                  [126.36544686061058, 72.34758068744891],
                  [126.3627861092678, 72.34791904122308],
                  [126.3613269875637, 72.34692998946569],
                  [126.35815125209007, 72.34487363120185],
                  [126.35737877589378, 72.34318151695544],
                  [126.35797959071311, 72.34164546193192]]),
  'scale': 1000,
  'crs': 'EPSG:46',
  'bestEffort': True,
  'maxPixels': 1e14,
  'tileScale': 4}

In [244]:
Map.centerObject(MESH1, 5)

In [245]:
train = ee.FeatureCollection('/Users/user/Downloads/LenaDelta_Vegetation.kmz')

In [252]:
# Display the train/test dataset
#center = train.geometry().centroid().getInfo()['coordinates']
#center.reverse()

In [255]:
import geopandas as gpd
lenaDelta = ee.FeatureCollection(Map.draw_features)
#selected_states = states.filterBounds(roi)
#Map.addLayer(selected_states, {}, "Selected states")

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
LenaDelta = os.path.join(out_dir, 'lenaDelta.shp')  


#out_landsat_stats = os.path.join(out_dir, 'landsat_stats.csv') 

ModuleNotFoundError: No module named 'geopandas'